In [ ]:
import sys
sys.path.append("../")
sys.path.append("../src")
sys.path.append("../hyena-dna/")

import numpy as np

from reglm.dataset import CharDataset

from reglm.	lightning_llama import CharLanguageDataset

In [ ]:
import numpy as np
import pandas as pd
import torch
from plotnine import *

## Download yeast native promoter expression data

yeast_data_train = pd.read_csv("../../dnadesign/data_and_model/mdlm/gosai_data/dataset_threecellline_train_crosschr.csv")
yeast_data_val = pd.read_csv("../../dnadesign/data_and_model/mdlm/gosai_data/dataset_threecellline_val_crosschr.csv")


In [ ]:
yeast_data_val

In [ ]:
train = yeast_data_train
val = yeast_data_val

In [ ]:
train_new = pd.DataFrame()
val_new = pd.DataFrame()

## Bin the sequences and create labels

In [ ]:
inst = []
seq = []
value_l = []
for item in train.index:
    row = train.loc[item]
    inst.append(f"Please generate a cis-regulatory element from chromosome {row['chrom']} with expression level {round(row['hepg2'],3)} in hepg2: ")
    inst.append(f"Please generate a cis-regulatory element from chromosome {row['chrom']} with expression level {round(row['k562'],3)} in k562: ")
    inst.append(f"Please generate a cis-regulatory element from chromosome {row['chrom']} with expression level {round(row['sknsh'],3)} in sknsh: ")
    seq.append(row['seq'])
    seq.append(row['seq'])
    seq.append(row['seq'])
    
    value_l.append(round(row['hepg2'],3))
    value_l.append(round(row['k562'],3))
    value_l.append(round(row['sknsh'],3))
train_new['instruction'] = inst
train_new['output'] = seq
train_new['exp'] = value_l

inst = []
seq = []
value_l = []
for item in val.index:
    row = val.loc[item]
    inst.append(f"Please generate a cis-regulatory element from chromosome {row['chrom']} with expression level {round(row['hepg2'],3)} in hepg2: ")
    inst.append(f"Please generate a cis-regulatory element from chromosome {row['chrom']} with expression level {round(row['k562'],3)} in k562: ")
    inst.append(f"Please generate a cis-regulatory element from chromosome {row['chrom']} with expression level {round(row['sknsh'],3)} in sknsh: ")
    seq.append(row['seq'])
    seq.append(row['seq'])
    seq.append(row['seq'])
    
    value_l.append(round(row['hepg2'],3))
    value_l.append(round(row['k562'],3))
    value_l.append(round(row['sknsh'],3))
val_new['instruction'] = inst
val_new['output'] = seq
val_new['exp'] = value_l

In [ ]:
train_new = train_new.sample(frac=1, random_state=2024)
# val_new = val_new.sample(frac=1, random_state=2024)

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
import pandas as pd

In [ ]:
import reglm

In [ ]:
# import PyPDF2
import os
import torch
os.environ['HF_HOME'] = '/home/tl688/scratch/'
os.environ['HF_TOKEN'] = 'fill'
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
instruct_len = 30
from llama_text_pred import CharExpLanguageDataset #llama_text means there is no predictor

In [ ]:
# train_ds = reglm.lightning_llama_edit.CharLanguageDataset(train['output'].values, train['instruction'].values, llm_name="Qwen/Qwen2.5-7B-Instruct")
# val_ds = reglm.lightning_llama_edit.CharLanguageDataset(val['output'].values, val['instruction'].values, llm_name="Qwen/Qwen2.5-7B-Instruct")

train_ds = CharExpLanguageDataset(train_new['output'].values, train_new['instruction'].values, train_new['exp'].values, llm_name="Qwen/Qwen2.5-3B-Instruct", max_token_len=instruct_len)
val_ds = CharExpLanguageDataset(val_new['output'].values, val_new['instruction'].values, train_new['exp'].values, llm_name="Qwen/Qwen2.5-3B-Instruct", max_token_len=instruct_len)

In [ ]:
# import hyenadna

In [ ]:
config = {
 'd_model': 32,
 'n_layer': 2,
 'd_inner': 32,
 'vocab_size': 12,
 'pad_vocab_size_multiple': 8,
 'return_hidden_state': True,
 # 'layer': {
 #     'emb_dim': 5,
 #     'filter_order': 64,
 #     'l_max': train_ds.seq_len + train_ds.label_len + 1,
 #     '_name_': 'hyena'
 # }
}

In [ ]:
HYENADNA_MODEL_NAMES = [
    "hyenadna-tiny-16k-seqlen-d128",
    "hyenadna-large-1m-seqlen",
    "hyenadna-medium-160k-seqlen",
    "hyenadna-medium-450k-seqlen",
    "hyenadna-small-32k-seqlen",
    "hyenadna-tiny-1k-seqlen",
    "hyenadna-tiny-1k-seqlen-d256",
]

model_name = "hyenadna-tiny-1k-seqlen-d256"
# model_name = "hyenadna-tiny-16k-seqlen-d128"

In [ ]:
reglm

In [ ]:
from llama_text_pred import LightningModel

In [ ]:

model = LightningModel(
    config=config, lr=1e-3, label_len=1,  ckpt_dir=f"./checkpoints/{model_name}", save_dir='./generation_more_test_value_align_qwen2.53b100epoch_3cl_crosschr_predexp/',llm_name="Qwen/Qwen2.5-3B-Instruct",latent_size=2048,align_head='relu', max_token_len=instruct_len)

In [ ]:
for param in model.Llama_encoder.parameters():
    param.requires_grad = False

## Train and validate model

In [ ]:
trainer = model.train_on_dataset(train_ds, val_ds, max_epochs=1000,
            batch_size=1024, num_workers=2, device=0, val_check_interval=700,
        )

In [ ]:
trainer.checkpoint_callback.best_model_path

In [ ]:
model = LightningModel.load_from_checkpoint(trainer.checkpoint_callback.best_model_path, llm_name = "Qwen/Qwen2.5-3B-Instruct",
        latent_size = 2048, max_token_len=instruct_len)

model = model.to(torch.device(0))

In [ ]:
import reglm.metrics

In [ ]:
import reglm.metrics

In [ ]:
test = pd.read_csv("../../dnadesign/data_and_model/mdlm/gosai_data/dataset_threecellline_test_crosschr.csv")

test_new = pd.DataFrame()

In [ ]:
test.columns

In [ ]:
# !nvidia-smi

In [ ]:
test_new = pd.DataFrame()
np.random.seed(2024)
inst = []
seq = []
for item in test.index:
    row = test.loc[item]
    random_choice_result = np.random.choice(['hepg2', 'k562', 'sknsh'])
    inst.append(f"Please generate a cis-regulatory element from chromosome {row['chrom']} with expression level {round(row[random_choice_result],3)} in {random_choice_result}: ")
    seq.append(row['seq'])

test_new['instruction'] = inst
test_new['output'] = seq

test_new

In [ ]:
test_new = test_new.sample(n=5000, random_state=2024)

In [ ]:

from tqdm import tqdm

generated_result = []

for item in tqdm(range(0,len(test_new),1024)):
    prompts = list(test_new['instruction'].values[item:item+1024])
    generated_result += model.generate(prompts, max_new_tokens=200, top_k=4, seed=item)
#     break

pd.DataFrame(generated_result).to_csv("3cl_qwem2.5_3b_finaldata_withregressor_updateepoch2.csv")